In [ ]:
%%capture

## Requires pip install the following packages
## - openai
## - tabulate
%pip install --upgrade openai tabulate

In [ ]:
import os
import openai
openai.organization = os.getenv('BRAINSTORMER_OPENAI_ORG')
openai.api_key = os.getenv('BRAINSTORMER_OPENAI_KEY')

assert openai.Model.list() is not None

In [ ]:
#topic = "Computer game about \"Harvest\""
topic = "How to teach a monkey do things"
#previous = [
#    "Indoor Treasure Hunt | Create a treasure hunt with clues and small prizes hidden throughout the house. Children will have a great time searching for the hidden treasure and solving the clues.",
#    "DIY Playdough | Make homemade playdough with flour, salt, water, and food coloring. Kids will love mixing the ingredients together and then playing with the colorful playdough.",
#    "Movie Marathon | Choose a few of your favorite movies and have a movie marathon with the kids. Don't forget the popcorn and snacks!",
#]
previous = [
    "Monkey-Proof Your Home | Make sure your house is monkey-proofed to avoid any damage or messes.",
    "Teach Monkey to do Tricks | Train the monkey to do tricks such as jumping through hoops, balancing on a ball, or riding a bike."
]

In [ ]:
previous_list = ", ".join(["\"{}\"".format(p.split("|")[0].strip()) for p in previous]) if previous and len(previous) > 1 else None

previous_text = """
Given this list of already selected ideas we want to elaborate on:
title | description
{}
""".format("\n".join(previous)) if previous and len(previous) > 1 else ""


In [ ]:
if previous and len(previous) > 1:
    question_text = "The next best 5 ideas that based combine ideas from {} are:".format(previous_list)
else:
    question_text = "The next best 5 ideas are:"

prompt = """
# Brainstorming session

This document keeps track of possible ideas.

The ideas will be stored in a well formed CSV document. Where the columns will be separated by the character |.

The document will have a list of ideas and every idea will have the columns "title" with a name for the idea and "description" with the idea description.

For example with the topic: "Activities with children for a rainy day" an example list of ideas could be:

```
title | description
Indoor Treasure Hunt | Create a treasure hunt with clues and small prizes hidden throughout the house. Children will have a great time searching for the hidden treasure and solving the clues.
DIY Playdough | Make homemade playdough with flour, salt, water, and food coloring. Kids will love mixing the ingredients together and then playing with the colorful playdough.
Movie Marathon | Choose a few of your favorite movies and have a movie marathon with the kids. Don't forget the popcorn and snacks!
```
## New Ideas for {}

{}

{}

title | description
""".format(topic, previous_text, question_text)

In [ ]:
completion = openai.Completion.create(
    model="text-davinci-003",
    presence_penalty=1,
    temperature=0.6,
    top_p=1,
    best_of=1,
    frequency_penalty=1,
    max_tokens=512,
    prompt=prompt)
print(completion.choices[0].text)

In [ ]:
more_prompt = prompt+completion.choices[0].text+"\n\nOther 5 ideas that are much crazier:\ntitle | description\n"

In [ ]:
completion = openai.Completion.create(
    model="text-davinci-003",
    presence_penalty=1,
    temperature=0.6,
    top_p=1,
    best_of=1,
    frequency_penalty=1,
    max_tokens=512,
    prompt=more_prompt)
print(completion.choices[0].text)

In [ ]:
[{"title": a.strip(), "description": b.strip()} for a, b in [line.split("|") for line in completion.choices[0].text.split("\n")]]